In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model

# Load the HMDB51 dataset
(ds_train, ds_test), ds_info = tfds.load(
    'hmdb51',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    as_supervised=True
)

# Preprocessing function for the dataset
def preprocess(video, label):
    video = tf.image.resize(video, (64, 64)) # Resize the video
    video = video / 255.0 # Normalize the video
    return video, label

# Preprocess the datasets
ds_train = ds_train.map(preprocess).batch(8).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(8).prefetch(tf.data.experimental.AUTOTUNE)



DatasetNotFoundError: Dataset hmdb51 not found.
Available datasets:
	- abstract_reasoning
	- accentdb
	- aeslc
	- aflw2k3d
	- ag_news_subset
	- ai2_arc
	- ai2_arc_with_ir
	- amazon_us_reviews
	- anli
	- answer_equivalence
	- arc
	- asqa
	- asset
	- assin2
	- bair_robot_pushing_small
	- bccd
	- beans
	- bee_dataset
	- beir
	- big_patent
	- bigearthnet
	- billsum
	- binarized_mnist
	- binary_alpha_digits
	- ble_wind_field
	- blimp
	- booksum
	- bool_q
	- bucc
	- c4
	- caltech101
	- caltech_birds2010
	- caltech_birds2011
	- cardiotox
	- cars196
	- cassava
	- cats_vs_dogs
	- celeb_a
	- celeb_a_hq
	- cfq
	- cherry_blossoms
	- chexpert
	- cifar10
	- cifar100
	- cifar100_n
	- cifar10_1
	- cifar10_corrupted
	- cifar10_n
	- citrus_leaves
	- cityscapes
	- civil_comments
	- clevr
	- clic
	- clinc_oos
	- cmaterdb
	- cnn_dailymail
	- coco
	- coco_captions
	- coil100
	- colorectal_histology
	- colorectal_histology_large
	- common_voice
	- conll2003
	- controlled_noisy_web_labels
	- coqa
	- cos_e
	- 

In [ ]:

# Create a model architecture
def create_model(input_shape, num_classes):
    inputs = Input(input_shape)
    x = Conv3D(32, (3, 3, 3), activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Conv3D(64, (3, 3, 3), activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model parameters
input_shape = (64, 64, None, 3)
num_classes = ds_info.features['label'].num_classes

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(ds_train, epochs=10, validation_data=ds_test)